# Prefiltering with [Gemini 2.0-flash](https://ai.google.dev/gemini-api/docs/quickstart?hl=fr)

According to the [benchmark](./../../benchmarking/benchmark_summary.ipynb), gemini 2.0-flash achieves an accuracy of about 87%. Therefore, we will use it to prefilter [subsets](./../../data/subsets_Di/) to gather more toxic contents.

## Libraries

In [1]:
from google import genai
from time import sleep 
from pathlib import Path
import os
import pandas as pd
from tqdm.rich import tqdm
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score
)
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.text import Text
import warnings
from google.api_core import exceptions as genai_errors
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

tqdm.pandas(desc="Fetching moderation scores")
console = Console()
warnings.filterwarnings("ignore")

## Global variables

In [2]:
ROOT = Path("../..")
DATA_DIR = ROOT / "data"
range_authorized = (6, 8) # (a,b) -> [a, a+1, ..., b-1]
subsets = [f for f in os.listdir(DATA_DIR / "subsets_Di") if f.replace(".csv","").replace("subset_", "") in map(str, range(range_authorized[0], range_authorized[1]))]
output_path = DATA_DIR / "pre-filtering" / f"pre-filtered_{range_authorized[0]}_{range_authorized[1]}.csv"
API_KEY_PATH = DATA_DIR / "confidential" / "GEMINI_API.txt"
console = Console()
model = "gemini-2.0-flash"
system_prompt = (Path("../../benchmarking") / "API_SYSTEM_PROMPT.txt").read_text().strip()
prompt = (Path("../../benchmarking") / "API_PROMPT.txt").read_text().strip()

In [3]:
os.environ["HTTP_PROXY"] = "socks5h://127.0.0.1:1080"
os.environ["HTTPS_PROXY"] = "socks5h://127.0.0.1:1080"

In [4]:
client = genai.Client(api_key=API_KEY_PATH.read_text().strip())

## Load dataset

In [5]:
dfs = [pd.read_csv(DATA_DIR / "subsets_Di" / f, encoding='utf-8') for f in subsets]
df = pd.concat(dfs, ignore_index=True)
df

,msg_id,user,content,topic,deleted,banned,hour
0,anon_msg_bb9a332e30a3,anon_user_85d33e9c85,"Niveau 17, mais pas de place de gagné dans le ...",anon_topic_720a2ca2,1,0,1
1,anon_msg_0df96db4eb86,anon_user_f8acb4acc9,première fois de ma vie que j'active cette fou...,anon_topic_a4ce6e29,1,0,1
2,anon_msg_0d0c4e9a05c1,anon_user_a0255d6952,Donc pour toi louer une voiture avec de l'arge...,anon_topic_363ba1aa,1,0,12
3,anon_msg_249e9dda9365,anon_user_80afdf5e58,Mélinda qui pleure qui n'était qu'une invocati...,anon_topic_bb3c1e80,1,1,14
4,anon_msg_2d1eccf5e6a0,anon_user_2eecf5800a,Il est où le S\n:ouch:,anon_topic_47c25947,1,0,14
...,...,...,...,...,...,...,...
59995,anon_msg_2868afd5a685,anon_user_4be6baa814,"Il a raison Pada, ça pue vraiment la merde cet...",anon_topic_afe95916,1,0,1
59996,anon_msg_d4f633ace9db,anon_user_8d0032111f,T'es toujours debout\nSanté mon clé\n:oui:\nPe...,anon_topic_76a84150,0,0,1
59997,anon_msg_34a099dc26ea,anon_user_df6545a6c3,"""Ton égo est en miettes ça te dit d'aller nour...",anon_topic_7a185bea,1,0,11
59998,anon_msg_68679115d5f8,anon_user_0a624ffe86,"Les meufs qui savent qu'elles nous plaisent, q...",anon_topic_d2868aa9,0,0,2


## Define prediction function

In [6]:
def predict(text, max_retries=5, delay_base=2):
    input_text = system_prompt + "\n" + prompt + f"« {text} »" + "\n Ce message est-il toxique ?\n"
    for attempt in range(max_retries):
        try:
            response = client.models.generate_content(
                model=model,
                contents=input_text
            )
            result = response.text.strip().lower()
            if result == "toxique":
                return 1
            elif result == "non-toxique":
                return 0
            else:
                console.print(f"[red]Unexpected response: '{result}'[/red]")
                return None
        except genai_errors.ServiceUnavailable as e:
            wait_time = delay_base ** attempt + random.uniform(0, 1)
            console.print(f"[yellow]Model overloaded (attempt {attempt + 1}/{max_retries}). Retrying in {wait_time:.2f}s...[/yellow]")
            sleep(wait_time)
        except Exception as e:
            wait_time = delay_base ** attempt + random.uniform(0, 1)
            console.print(f"[red]Error: {e}[/red]")
            sleep(wait_time)
    console.print(f"[red]Max retries reached for: {text}[/red]")
    return None

def safe_predict(text):
    try:
        return predict(text)
    except Exception as e:
        console.print(f"[red]Error for text: {text} → {e}[/red]")
        return None

## Run prediction

In [7]:
# List of texts to classify
texts = df["content"].tolist()

# before launching:
results = [None] * len(texts)

checkpoint_path = output_path.with_suffix(".checkpoint.csv")

with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_idx = {
        executor.submit(safe_predict, txt): idx
        for idx, txt in enumerate(texts)
    }
    for future in tqdm(as_completed(future_to_idx), total=len(texts)):
        idx = future_to_idx[future]
        results[idx] = future.result()
        if idx % 500 == 0:
            pd.DataFrame({"content": texts, "label": results}).to_csv(checkpoint_path, index=False)
            console.print(f"[blue]Checkpoint saved at {checkpoint_path}[/blue]")

Output()

Checkpoint saved at ../../data/pre-filtering/pre-filtered_6_8.checkpoint.csv

Checkpoint saved at ../../data/pre-filtering/pre-filtered_6_8.checkpoint.csv

Checkpoint saved at ../../data/pre-filtering/pre-filtered_6_8.checkpoint.csv

Checkpoint saved at ../../data/pre-filtering/pre-filtered_6_8.checkpoint.csv

...

Error: 502 Bad Gateway. {'message': '<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport
content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 502 (Server Error)!!1</title>\n  
<style>\n    *{margin:0;padding:0}html,code{font:15px/22px 
arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 
0;max-width:390px;min-height:180px;padding:30px 0 15px}* > 
body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 
0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and 
(max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.goog
le.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen 
and 
(min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54
dp.png) no-repeat 0% 0%/100% 
100%;-moz-border-image:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) 0}}@media 
only screen and 
(-webkit-min-device-pixel-ratio:2){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_c
olor_150x54dp.png) no-repeat;-webkit-background-size:100% 
100%}}#logo{display:inline-block;height:54px;width:150px}\n  </style>\n  <a href=//www.google.com/><span id=logo 
aria-label=Google></span></a>\n  <p><b>502.</b> <ins>That’s an error.</ins>\n  <p>The server encountered a 
temporary error and could not complete your request.<p>Please try again in 30 seconds.  <ins>That’s all we 
know.</ins>\n', 'status': 'Bad Gateway'}

Error: 502 Bad Gateway. {'message': '<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport
content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 502 (Server Error)!!1</title>\n  
<style>\n    *{margin:0;padding:0}html,code{font:15px/22px 
arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 
0;max-width:390px;min-height:180px;padding:30px 0 15px}* > 
body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 
0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and 
(max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.goog
le.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen 
and 
(min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54
dp.png) no-repeat 0% 0%/100% 
100%;-moz-border-image:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) 0}}@media 
only screen and 
(-webkit-min-device-pixel-ratio:2){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_c
olor_150x54dp.png) no-repeat;-webkit-background-size:100% 
100%}}#logo{display:inline-block;height:54px;width:150px}\n  </style>\n  <a href=//www.google.com/><span id=logo 
aria-label=Google></span></a>\n  <p><b>502.</b> <ins>That’s an error.</ins>\n  <p>The server encountered a 
temporary error and could not complete your request.<p>Please try again in 30 seconds.  <ins>That’s all we 
know.</ins>\n', 'status': 'Bad Gateway'}

Error: 502 Bad Gateway. {'message': '<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport
content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 502 (Server Error)!!1</title>\n  
<style>\n    *{margin:0;padding:0}html,code{font:15px/22px 
arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 
0;max-width:390px;min-height:180px;padding:30px 0 15px}* > 
body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 
0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and 
(max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.goog
le.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen 
and 
(min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54
dp.png) no-repeat 0% 0%/100% 
100%;-moz-border-image:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) 0}}@media 
only screen and 
(-webkit-min-device-pixel-ratio:2){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_c
olor_150x54dp.png) no-repeat;-webkit-background-size:100% 
100%}}#logo{display:inline-block;height:54px;width:150px}\n  </style>\n  <a href=//www.google.com/><span id=logo 
aria-label=Google></span></a>\n  <p><b>502.</b> <ins>That’s an error.</ins>\n  <p>The server encountered a 
temporary error and could not complete your request.<p>Please try again in 30 seconds.  <ins>That’s all we 
know.</ins>\n', 'status': 'Bad Gateway'}

Checkpoint saved at ../../data/pre-filtering/pre-filtered_6_8.checkpoint.csv

Checkpoint saved at ../../data/pre-filtering/pre-filtered_6_8.checkpoint.csv

In [8]:
df['gemini_prediction'] = results

In [9]:
for i, row in df.sample(5, random_state=42).iterrows():
    content = Text(row['content'], style="bold")
    toxicity = f"[yellow]Gemini Prediction:[/yellow] [bold]{int(row['gemini_prediction'])}[/bold]"
    panel = Panel.fit(
        f"{content}\n\n{toxicity}",
        title=f"Exemple {i+1}",
        border_style="magenta"
    )
    console.print(panel)

╭────────────────────────── Exemple 12629 ───────────────────────────╮
│ bonne chance pour l'avenir perso encore deux ans ici après je part │
│                                                                    │
│ Gemini Prediction: 0                                               │
╰────────────────────────────────────────────────────────────────────╯

╭───────── Exemple 37731 ─────────╮
│ J'ai jamais été un gros dormeur │
│ Ma moyenne c'est 4H par nuit    │
│                                 │
│ Gemini Prediction: 0            │
╰─────────────────────────────────╯

╭─────────────────────────────────────────── Exemple 39992 ───────────────────────────────────────────╮
│ Les famosos gilets jaunes dissidents mais triples dosés, masqués et soumis au pa-pass sans broncher │
│ Bons esclaves prêts à retourner travailler lundi                                                    │
│                                                                                                     │
│ Gemini Prediction: 1                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────── Exemple 8526 ─────────────────────╮
│ J'irais en Chine rien que pour rencontrer cette déesse │
│ :coeur:                                                │
│                                                        │
│ Gemini Prediction: 0                                   │
╰────────────────────────────────────────────────────────╯

╭──────────── Exemple 8280 ─────────────╮
│ Non avec mon AAH j’ai pas de livret A │
│                                       │
│ Gemini Prediction: 0                  │
╰───────────────────────────────────────╯

In [10]:
df['gemini_prediction'].value_counts()

gemini_prediction
0    46336
1    13664
Name: count, dtype: int64

In [11]:
df.to_csv(output_path, index=False, encoding="utf-8")